# 市区町村別外国人比率バブルチャート

- **横軸**: 外国人比率（%）
- **縦軸**: 前年比（%）
- **バブルサイズ**: 外国人人口
- **データ**: 住民基本台帳（最新年と前年の比較）

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path

DATA_DIR = Path('.')
df = pd.read_csv(DATA_DIR / 'daicho_estat.csv')

In [ ]:
# 最新年・前年を取得
max_year = df['year'].max()
prev_year = max_year - 1
print(f'最新年: {max_year}, 前年: {prev_year}')

df_cur = df[df['year'] == max_year].copy()
df_prv = df[df['year'] == prev_year][['団体コード', '外国人人口']].rename(columns={'外国人人口': '外国人_前年'})

# マージして比率・前年比を計算
df_m = df_cur.merge(df_prv, on='団体コード', how='left')
df_m['外国人比率'] = (df_m['外国人人口'] / df_m['総人口'] * 100).round(2)
df_m['前年比'] = ((df_m['外国人人口'] - df_m['外国人_前年']) / df_m['外国人_前年'] * 100).round(2)

# 欠損・異常値除外
df_plot = df_m.dropna(subset=['外国人比率', '前年比']).copy()
df_plot = df_plot[df_plot['外国人人口'] > 0]
print(f'プロット対象: {len(df_plot):,}件')
df_plot[['都道府県名', '市区町村名', '外国人比率', '前年比', '外国人人口']].head()

In [ ]:
# 都道府県ごとにトレースを分けてカラーリング
PREF_ORDER = [
    '北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県',
    '茨城県', '栃木県', '群馬県', '埼玉県', '千葉県', '東京都', '神奈川県',
    '新潟県', '富山県', '石川県', '福井県', '山梨県', '長野県',
    '岐阜県', '静岡県', '愛知県', '三重県',
    '滋賀県', '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県',
    '鳥取県', '島根県', '岡山県', '広島県', '山口県',
    '徳島県', '香川県', '愛媛県', '高知県',
    '福岡県', '佐賀県', '長崎県', '熊本県', '大分県', '宮崎県', '鹿児島県', '沖縄県',
]

import plotly.colors
colors = plotly.colors.qualitative.Dark24 + plotly.colors.qualitative.Light24
pref_color = {p: colors[i % len(colors)] for i, p in enumerate(PREF_ORDER)}

fig = go.Figure()

for pref in PREF_ORDER:
    d = df_plot[df_plot['都道府県名'] == pref]
    if d.empty:
        continue
    fig.add_trace(go.Scatter(
        x=d['外国人比率'],
        y=d['前年比'],
        mode='markers',
        name=pref,
        marker=dict(
            size=6,
            color=pref_color.get(pref, '#888'),
            opacity=0.65,
            line=dict(width=0.5, color='white'),
        ),
        text=d['市区町村名'],
        customdata=d[['外国人人口', '総人口']].values,
        hovertemplate=(
            '<b>%{text}</b> (%{fullData.name})<br>'
            '外国人比率: %{x:.2f}%<br>'
            '前年比: %{y:+.2f}%<br>'
            '外国人人口: %{customdata[0]:,}人<br>'
            '総人口: %{customdata[1]:,}人<extra></extra>'
        ),
    ))

# ゼロライン
fig.add_hline(y=0, line=dict(color='gray', width=1, dash='dot'))

fig.update_layout(
    title=f'市区町村別外国人比率散布図（{max_year}年1月, 前年比は{prev_year}年比）',
    xaxis=dict(title='外国人比率（%）', showgrid=True, gridcolor='#eee', zeroline=False),
    yaxis=dict(title='前年比（%）', showgrid=True, gridcolor='#eee'),
    legend=dict(
        title='都道府県',
        orientation='v',
        x=1.01, y=1,
        font=dict(size=10),
    ),
    height=700,
    margin=dict(l=60, r=180, t=60, b=60),
    plot_bgcolor='white',
    hovermode='closest',
)

fig.show()

In [ ]:
# 参考: 前年比の外れ値確認
print('前年比 上位10件')
print(df_plot.nlargest(10, '前年比')[['都道府県名', '市区町村名', '外国人比率', '前年比', '外国人人口']].to_string(index=False))
print()
print('前年比 下位10件')
print(df_plot.nsmallest(10, '前年比')[['都道府県名', '市区町村名', '外国人比率', '前年比', '外国人人口']].to_string(index=False))

In [ ]:
# 外国人比率 上位20件
print('外国人比率 上位20件')
print(df_plot.nlargest(20, '外国人比率')[['都道府県名', '市区町村名', '外国人比率', '前年比', '外国人人口']].to_string(index=False))